In [4]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import r2_score
pd.set_option('display.float_format', lambda x: '%.10f' % x)
from sklearn.metrics import confusion_matrix
import statsmodels.graphics.tsaplots as sg
from statsmodels.tsa.stattools import adfuller
from sklearn.mixture import GaussianMixture 
pd.set_option('display.float_format', lambda x: '%.10f' % x)
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score

%matplotlib inline

In [5]:
RecordWritingPath = '/Users/hemingyi/Documents/capstone/post/result/'
TransportationDataPath = '/Users/hemingyi/Documents/capstone/post/transportation/'
EventDataPath = '/Users/hemingyi/Documents/capstone/post/events/'
comboPath = '/Users/hemingyi/Documents/capstone/combo/'
PostData = '/Users/hemingyi/Documents/capstone/post/'
# dataFile = TransportationDataPath+city+'EdgeYearwiseAggregated.csv'

In [6]:
def anomalyDetection(y,nc, pval = 0.2,iterN=20):
    if len(y.shape) == 1:
        y = np.array(y).reshape(-1,1)
    rind = np.array(range(y.shape[0]))
    #clustering model
    gm=GaussianMixture(n_components=nc,n_init=100,max_iter=1000,random_state=0) 
    for i in range(iterN):
#         print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if all(rind==rind0):
#             print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres


def detectOutlier(df_residuals, df_finalEvents, nc):
    df_results = pd.DataFrame(columns= ['Thres', 'F1'] + list(df_finalEvents['Type'].unique()) + ['tn', 'fp', 'fn', 'tp'])
    eventsize = len(df_finalEvents['Date'].unique())/ len(df_residuals)
    a = 0
    for thres in [eventsize] + list(np.arange(0.1,1.0,0.1)):
#         print(thres)
        lis = [thres]
        df_residuals['Outliers'] = anomalyDetection(df_residuals['residuals'],nc=nc,pval = thres)
        lis.append(f1_score(df_residuals['true'], df_residuals['Outliers']))
        for event in df_finalEvents['Type'].unique():
            df_resTemp = df_residuals[df_residuals.index.isin(df_finalEvents[df_finalEvents['Type']==event]['Date'])]
            lis.append(np.sum(df_resTemp['Outliers'])/len(df_resTemp))
        lis = lis  + list(confusion_matrix(df_residuals['true'], df_residuals['Outliers']).ravel())
        #print(lis)
        df_results.loc[a,:] = lis
        a = a+1
    return df_results

In [7]:
df_results = pd.DataFrame()
for city in ['DC', 'Taipei', 'NewYork', 'Chicago']:
    print(city)
    subway_data = TransportationDataPath+city+'EdgeDatewiseAggregated.csv'
    events_data = EventDataPath+city+'Events.csv'

    # import data
    df_main = pd.read_csv(subway_data)

    # Aggregate on date
    df_main = pd.DataFrame(df_main.groupby(['date'])['amount'].sum())

    # sort by index
    df_main = df_main.sort_index()
    

    # print details
    print(df_main.info())

    # change to datetime format
    df_main.index = pd.to_datetime(df_main.index)
#     print(df_main)
#     break
    # import events data
    df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)

    # dataframe for events
    df_finalEvents =  df_events[['Date', 'Type']]
    df_finalEvents.index = pd.to_datetime(df_finalEvents.Date)
    df_output = df_main.join(df_finalEvents, how='left')
    del df_output['Date']
#     df_output.to_csv(WebData+city+'DailyAggregated.csv')
#     print(df_output)
#     break

    # list events
    lis_event = df_finalEvents['Type'].unique()
    lis_event = list(lis_event)

    # fit model
    model = ARIMA(df_main['amount'], order=(7,1,2))
    model_fit = model.fit(disp=0)
#     print(model_fit.summary())

    # create residuals and predictions
    df_residuals = pd.DataFrame(model_fit.resid, columns=['residuals'])

    ## Original Anomalies
    df_residuals['true'] = False
    df_residuals.loc[df_residuals.index.isin(df_finalEvents['Date'].unique()), 'true'] = True

    for i in range(1,6):
        df_resultsTemp = detectOutlier(df_residuals, df_finalEvents, i)
        df_resultsTemp['n_comp'] = i
        df_resultsTemp['city'] = city
        df_results = df_results.append(df_resultsTemp)
    

DC
<class 'pandas.core.frame.DataFrame'>
Index: 725 entries, 2016-01-01 to 2017-12-31
Data columns (total 1 columns):
amount    725 non-null int64
dtypes: int64(1)
memory usage: 11.3+ KB
None
Taipei
<class 'pandas.core.frame.DataFrame'>
Index: 638 entries, 2017-01-01 to 2018-09-30
Data columns (total 1 columns):
amount    638 non-null float64
dtypes: float64(1)
memory usage: 10.0+ KB
None
NewYork
<class 'pandas.core.frame.DataFrame'>
Index: 579 entries, 2017-06-01 to 2018-12-31
Data columns (total 1 columns):
amount    579 non-null int64
dtypes: int64(1)
memory usage: 9.0+ KB
None
Chicago
<class 'pandas.core.frame.DataFrame'>
Index: 731 entries, 01/01/2015 to 12/31/2016
Data columns (total 1 columns):
amount    731 non-null float64
dtypes: float64(1)
memory usage: 11.4+ KB
None


In [8]:
df_results.to_csv(RecordWritingPath+'TSResults.csv')